In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [4]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of species : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [5]:
X = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(df["ImageFilePath"], df["ScientificName"])) ])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


11399it [26:42,  7.11it/s]


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 93.60% | Prec: 94.53% | Rec: 93.60% | F1: 93.67% | Bal Acc: 81.43% | MCC: 0.9318


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 99.39% | Prec: 99.42% | Rec: 99.39% | F1: 99.35% | Bal Acc: 93.30% | MCC: 0.9934


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


...

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 96.10% | Prec: 95.78% | Rec: 96.10% | F1: 95.56% | Bal Acc: 77.16% | MCC: 0.9583


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 89.69% | Prec: 87.35% | Rec: 89.69% | F1: 86.79% | Bal Acc: 47.69% | MCC: 0.8898
MLP_Baseline              | Acc: 99.82% | Prec: 99.74% | Rec: 99.82% | F1: 99.78% | Bal Acc: 96.69% | MCC: 0.9981


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Agonum punctiforme,Agonum punctiforme,Agonum punctiforme,Cyclotrachelus torvus,Agonum punctiforme,Agonum punctiforme,Amara quenseli,Agonum punctiforme


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.935965,0.945299,0.935965,0.936732,0.814259,0.931819
LogisticRegression,LogisticRegression,0.99386,0.994154,0.99386,0.993519,0.933043,0.993444
SVMLinear,SVMLinear,0.99693,0.996583,0.99693,0.996491,0.952812,0.996722
SVMPolynomial,SVMPolynomial,0.857018,0.882959,0.857018,0.844093,0.514143,0.850981
SVMRadialBasis,SVMRadialBasis,0.973684,0.961197,0.973684,0.965947,0.797199,0.971915
NearestNeighbor,NearestNeighbor,0.960965,0.957816,0.960965,0.955633,0.771595,0.958319
RandomForest,RandomForest,0.89693,0.873498,0.89693,0.867908,0.476948,0.88981
MLP_Baseline,MLP_Baseline,0.998246,0.997377,0.998246,0.99778,0.966912,0.998127


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/27.ViLT-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/27.ViLT-linear-probing-species-metrics.csv", index=False)